In [13]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
import time
import warnings
warnings.filterwarnings('ignore')

In [14]:
column_names = list(range(0,7))
column_names.append('Classification')

df = pd.read_csv("seeds_dataset.txt", names = column_names, sep = '\t') 
df

,0,1,2,3,4,5,6,Classification
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1
...,...,...,...,...,...,...,...,...
205,12.19,13.20,0.8783,5.137,2.981,3.631,4.870,3
206,11.23,12.88,0.8511,5.140,2.795,4.325,5.003,3
207,13.20,13.66,0.8883,5.236,3.232,8.315,5.056,3
208,11.84,13.21,0.8521,5.175,2.836,3.598,5.044,3


In [15]:
df.dtypes

0                 float64
1                 float64
2                 float64
3                 float64
4                 float64
5                 float64
6                 float64
Classification      int64
dtype: object

In [16]:
#find number of null entries to see if there is a need to clean the dataset
df.isnull().sum()
#df = df.dropna()

0                 0
1                 0
2                 0
3                 0
4                 0
5                 0
6                 0
Classification    0
dtype: int64

In [17]:
df.nunique()

0                 193
1                 170
2                 186
3                 188
4                 184
5                 207
6                 148
Classification      3
dtype: int64

In [18]:
from sklearn import preprocessing 
label_encoder = preprocessing.LabelEncoder() 

df['Classification'] = label_encoder.fit_transform(df['Classification'])
df

,0,1,2,3,4,5,6,Classification
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,0
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,0
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,0
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,0
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,0
...,...,...,...,...,...,...,...,...
205,12.19,13.20,0.8783,5.137,2.981,3.631,4.870,2
206,11.23,12.88,0.8511,5.140,2.795,4.325,5.003,2
207,13.20,13.66,0.8883,5.236,3.232,8.315,5.056,2
208,11.84,13.21,0.8521,5.175,2.836,3.598,5.044,2


In [19]:
X = df.drop('Classification', axis=1)
y = df['Classification']

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [20]:
from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

tf.random.set_seed(42)
model = tf.keras.Sequential()

model.add(tf.keras.layers.Dense(128, activation='relu', input_shape = (7,)))
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(3, activation='softmax'))

model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(lr=0.001),
    metrics=["accuracy",
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]
)

from sklearn.model_selection import KFold
kf = KFold(n_splits=10, random_state =42, shuffle=True)

sum_acc_score = 0
sum_prec_score = 0
sum_recall_score = 0
sum_f1_score = 0


for train_index, test_index in kf.split(X):
    X_train = pd.DataFrame()
    y_train = pd.DataFrame()

    X_test = pd.DataFrame()
    y_test = pd.DataFrame()
    
    for index in train_index:
        X_train = X_train.append(X.iloc[index])
        y_train = y_train.append(pd.Series(y.iloc[index]), ignore_index = True)
        
    for index in test_index:
        X_test = X_test.append(X.iloc[index])
        y_test = y_test.append(pd.Series(y.iloc[index]), ignore_index = True)

    train_X = tf.convert_to_tensor(X_train)
    test_X = tf.convert_to_tensor(X_test)

    #one hot encode y_train and y_test
    train_y = tf.keras.utils.to_categorical(y_train)
    test_y = tf.keras.utils.to_categorical(y_test)

            
    history = model.fit(train_X, train_y, epochs=50, validation_data = (test_X, test_y),callbacks=[callback])
    predictions = model.predict(test_X)
    
    predicted_classes = predictions.argmax(axis=1)
    test_y_classes = test_y.argmax(axis=1)
    
    sum_acc_score += accuracy_score(test_y_classes, predicted_classes)
    sum_prec_score += precision_score(test_y_classes, predicted_classes, average = 'micro')
    sum_recall_score += recall_score(test_y_classes, predicted_classes, average = 'micro')
    sum_f1_score += f1_score(test_y_classes, predicted_classes, average = 'micro')

Epoch 1/50
6/6 [==============================] - 4s 280ms/step - loss: 1.2495 - accuracy: 0.3439 - precision: 0.3824 - recall: 0.2751 - val_loss: 0.9579 - val_accuracy: 0.3810 - val_precision: 0.7500 - val_recall: 0.2857
Epoch 2/50
6/6 [==============================] - 0s 21ms/step - loss: 0.9601 - accuracy: 0.5185 - precision: 0.6250 - recall: 0.4233 - val_loss: 0.8216 - val_accuracy: 0.6667 - val_precision: 1.0000 - val_recall: 0.3333
Epoch 3/50
6/6 [==============================] - 0s 19ms/step - loss: 0.8551 - accuracy: 0.5979 - precision: 0.6932 - recall: 0.3228 - val_loss: 0.8043 - val_accuracy: 0.7619 - val_precision: 1.0000 - val_recall: 0.2857
Epoch 4/50
6/6 [==============================] - 0s 20ms/step - loss: 0.7382 - accuracy: 0.6931 - precision: 0.8958 - recall: 0.4550 - val_loss: 0.6972 - val_accuracy: 0.5238 - val_precision: 0.8462 - val_recall: 0.5238
Epoch 5/50
6/6 [==============================] - 0s 20ms/step - loss: 0.6931 - accuracy: 0.7513 - precision: 0.899

Epoch 38/50
6/6 [==============================] - 0s 12ms/step - loss: 0.2364 - accuracy: 0.9206 - precision: 0.9239 - recall: 0.8995 - val_loss: 0.3607 - val_accuracy: 0.9048 - val_precision: 0.9048 - val_recall: 0.9048
Epoch 39/50
6/6 [==============================] - 0s 15ms/step - loss: 0.2473 - accuracy: 0.9206 - precision: 0.9301 - recall: 0.9153 - val_loss: 0.2592 - val_accuracy: 0.8571 - val_precision: 0.8571 - val_recall: 0.8571
Epoch 40/50
6/6 [==============================] - 0s 20ms/step - loss: 0.2312 - accuracy: 0.9312 - precision: 0.9355 - recall: 0.9206 - val_loss: 0.2930 - val_accuracy: 0.8095 - val_precision: 0.8095 - val_recall: 0.8095
Epoch 41/50
6/6 [==============================] - 0s 15ms/step - loss: 0.2278 - accuracy: 0.9259 - precision: 0.9358 - recall: 0.9259 - val_loss: 0.2720 - val_accuracy: 0.8571 - val_precision: 0.8571 - val_recall: 0.8571
Epoch 42/50
6/6 [==============================] - 0s 16ms/step - loss: 0.2367 - accuracy: 0.9101 - precision: 0

6/6 [==============================] - 0s 10ms/step - loss: 0.2139 - accuracy: 0.9153 - precision: 0.9202 - recall: 0.9153 - val_loss: 0.1805 - val_accuracy: 0.9048 - val_precision: 0.9048 - val_recall: 0.9048
Epoch 11/50
1/1 [==============================] - 0s 30ms/step
Epoch 1/50
6/6 [==============================] - 0s 21ms/step - loss: 0.1832 - accuracy: 0.9153 - precision: 0.9149 - recall: 0.9101 - val_loss: 0.2284 - val_accuracy: 0.9048 - val_precision: 0.9048 - val_recall: 0.9048
Epoch 2/50
6/6 [==============================] - 0s 10ms/step - loss: 0.1921 - accuracy: 0.9153 - precision: 0.9202 - recall: 0.9153 - val_loss: 0.1852 - val_accuracy: 0.9048 - val_precision: 0.9048 - val_recall: 0.9048
Epoch 3/50
6/6 [==============================] - 0s 10ms/step - loss: 0.1847 - accuracy: 0.9206 - precision: 0.9255 - recall: 0.9206 - val_loss: 0.2257 - val_accuracy: 0.9048 - val_precision: 0.9048 - val_recall: 0.9048
Epoch 4/50
6/6 [==============================] - 0s 10ms/step 

6/6 [==============================] - 0s 12ms/step - loss: 0.1658 - accuracy: 0.9206 - precision: 0.9202 - recall: 0.9153 - val_loss: 0.2729 - val_accuracy: 0.9048 - val_precision: 0.9048 - val_recall: 0.9048
Epoch 12/50
6/6 [==============================] - 0s 13ms/step - loss: 0.1742 - accuracy: 0.9048 - precision: 0.9048 - recall: 0.9048 - val_loss: 0.2076 - val_accuracy: 0.9524 - val_precision: 0.9524 - val_recall: 0.9524
Epoch 13/50
6/6 [==============================] - 0s 13ms/step - loss: 0.1756 - accuracy: 0.9101 - precision: 0.9101 - recall: 0.9101 - val_loss: 0.2583 - val_accuracy: 0.9048 - val_precision: 0.9048 - val_recall: 0.9048
Epoch 14/50
1/1 [==============================] - 0s 45ms/step
Epoch 1/50
6/6 [==============================] - 0s 27ms/step - loss: 0.1675 - accuracy: 0.9312 - precision: 0.9309 - recall: 0.9259 - val_loss: 0.1979 - val_accuracy: 0.9048 - val_precision: 0.9000 - val_recall: 0.8571
Epoch 2/50
6/6 [==============================] - 0s 12ms/ste

6/6 [==============================] - 0s 9ms/step - loss: 0.2178 - accuracy: 0.8995 - precision: 0.8995 - recall: 0.8995 - val_loss: 0.2467 - val_accuracy: 0.9048 - val_precision: 0.9048 - val_recall: 0.9048
Epoch 9/50
6/6 [==============================] - 0s 12ms/step - loss: 0.2246 - accuracy: 0.8889 - precision: 0.8936 - recall: 0.8889 - val_loss: 0.1759 - val_accuracy: 0.8571 - val_precision: 0.9000 - val_recall: 0.8571
Epoch 10/50
6/6 [==============================] - 0s 12ms/step - loss: 0.1746 - accuracy: 0.9153 - precision: 0.9153 - recall: 0.9153 - val_loss: 0.1705 - val_accuracy: 0.9524 - val_precision: 0.9524 - val_recall: 0.9524
Epoch 11/50
6/6 [==============================] - 0s 12ms/step - loss: 0.1811 - accuracy: 0.9365 - precision: 0.9365 - recall: 0.9365 - val_loss: 0.2486 - val_accuracy: 0.9048 - val_precision: 0.9048 - val_recall: 0.9048
Epoch 12/50
6/6 [==============================] - 0s 12ms/step - loss: 0.2060 - accuracy: 0.9153 - precision: 0.9149 - recall

In [21]:
print('Average Accuracy Score: ', sum_acc_score/10)
print('Average Precision Score: ', sum_prec_score/10)
print('Average Recall Score: ', sum_recall_score/10)
print('Average F1-Score: ', sum_f1_score/10)



Average Accuracy Score:  0.9238095238095239
Average Precision Score:  0.9238095238095239
Average Recall Score:  0.9238095238095239
Average F1-Score:  0.9238095238095239
